In [ ]:
import tensorflow
import numpy
import prettytable
import networkx
import numba
import sklearn
import matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil

target_folder = ["sites", "output_data"]

if __name__ == "__main__":
    #root = os.getcwd()
    root = "/content/drive/MyDrive/"
    data_folder = os.path.join(os.path.dirname(root), "Floors/data")
    for dir_name in target_folder:
        dir_path = os.path.join(data_folder, dir_name)
        if os.path.isdir(dir_path):
            shutil.rmtree(dir_path)
        os.mkdir(dir_path)

In [ ]:
#import os
import json
#import sys
#sys.path.insert(0,"/content/drive/MyDrive/Floors")
from const import *

settings_id = "preprocess"
settings_node_id = "preprocess node"
settings_file = "/content/drive/MyDrive/Floors/settings.json"

if __name__ == "__main__":
    input_dir_name = ""
    output_dir_name = ""
    building_id = ""
    with open(settings_file, 'r') as f_in:
        all_settings = json.load(f_in)
        settings = all_settings[settings_id]
        input_dir_name = settings["raw input dir"]
        output_dir_name = settings["node output dir"]

    #root = os.getcwd()
    root = "/content/drive/MyDrive/Floors/"
    for building_id in building_ids:
        print (f"building_id: {building_id}")
        train_path = os.path.abspath(os.path.join(root, input_dir_name, building_id))
        for floor_id in os.listdir(train_path): # loop through each floor
            floor_path = os.path.abspath(os.path.join(train_path, floor_id))
            breakpoint_info_list = []
            for filename in os.listdir(floor_path): # loop through all files in each floor
                file_path = os.path.abspath(os.path.join(floor_path, filename))
                with open(file_path, 'rb') as f_in:
                    for line in f_in:
                        line_decoded = line.decode("utf-8")
                        if ('TYPE_WAYPOINT' in line_decoded):
                            waypoint_info = line_decoded.split('\t')
                            breakpoint_info_list.append([waypoint_info[2], waypoint_info[3].strip(), waypoint_info[0]])
            breakpoint_list = sorted(breakpoint_info_list, key=lambda x:int(x[2]))

            output_folder = os.path.abspath(os.path.join(root, output_dir_name, building_id))
            if not (os.path.exists(output_folder)):
                os.mkdir(output_folder)
            output_file = os.path.abspath(os.path.join(output_folder, "{}_WiFi.txt".format(floor_id)))

            with open(file_path, 'rb') as f_in, open(output_file, "a") as f_out:
                f_out.write("Start: \n") # first line
                for i in range(len(breakpoint_list)): # second line
                    f_out.write("{},{} ".format(breakpoint_list[i][0], breakpoint_list[i][1]))
                f_out.write("\n")
                for i in range(len(breakpoint_list)): # third line
                    f_out.write("{} ".format(breakpoint_list[i][2]))
                f_out.write("\n")

            current_time = 0
            first_time = True
            for filename in os.listdir(floor_path):
                with open(file_path, 'rb') as f_in, open(output_file, "a") as f_out:
                    for line in f_in:
                        line_decoded = line.decode("utf-8")
                        if ('TYPE_WIFI' in line_decoded):
                            wifi_info = line_decoded.split('\t')
                            if int(wifi_info[0])!=current_time:
                                if first_time:
                                    first_time = False
                                else:
                                    f_out.write("\n")
                                f_out.write("{} ".format(wifi_info[0]))
                                current_time = int(wifi_info[0])
                            f_out.write("{},{} ".format(wifi_info[2], wifi_info[4]))

In [ ]:
#copied from FYT/wifine
#import os
#import json
import pickle
#import shutil

#from utils import *
#from const import *

#utils.py module

def load_file_comparison(filename, observation_ids, observation_sign, file_type, prune):
    max_id = 0
    if observation_ids:
        max_id = max(max_id, int(observation_ids[-1][0][1:]))
    mac_all = []
    with open(filename, "r") as f_in:
        while True:
            line = f_in.readline().rstrip(" \n")
            if not line:
                break
            if file_type == "path":
                if line.startswith("Start:"):
                    device_id = os.path.basename(filename)[:2]
                    try:
                        breakpoints = [[float(coor) for coor in item.split(
                            ",")] for item in f_in.readline().rstrip(" \n").split(" ")]
                        timestamps = [
                            int(ts) for ts in f_in.readline().rstrip(" \n").split(" ")]
                    except ValueError:
                        break
                    continue
                timestamp, rssi_pairs = line.split(" ", 1)
                ground_truth = interpolate_point(
                    int(timestamp), timestamps, breakpoints)
            elif file_type == "db":
                device_id = os.path.basename(filename)
                coor, rssi_pairs = line.split(" ", 1)
                ground_truth = [float(item) for item in coor.split(",")]
            elif file_type == "new":
                wifi_json = json.loads(line)
                timestamp = wifi_json['sysTimeMs']
                rssi_pairs = ''
                for item in wifi_json['data']:
                    rssi_pairs += str(item['bssid'].replace(':','')) + ',' + str(item['rssi']) + ' '
                rssi_pairs = rssi_pairs.strip(' ')
                ground_truth = [None,None]
                device_id = None

            rssi_dict = {}
            for rssi_pair in rssi_pairs.split(" "):
                mac = rssi_pair.split(",")[0]
                rssi = rssi_pair.split(",")[1]
                # remove virtual mac
                if prune and is_virtual_mac(mac):
                    continue
                try:
                    rssi_dict[mac] = float(rssi)
                except ValueError:
                    continue
                if mac not in mac_all:
                    mac_all.append(mac)
            if rssi_dict:
                observation_ids.append(
                    ["{}{}".format(observation_sign, max_id+1), device_id, ground_truth, rssi_dict])
                max_id += 1

    print("{} loaded".format(filename))
    return observation_ids, mac_all


def series_to_file(obj, filename):
    with open(filename, 'wb') as f_out:
        pickle.dump(obj, f_out, -1)
        print("Data written into {}".format(filename))


def file_to_series(filename):
    with open(filename, 'rb') as f_in:
        series = pickle.load(f_in)
        print("File {} loaded.".format(filename))
        return series


def rssi2weight(offset, rssi):
    return offset + rssi


def is_virtual_mac(mac_addr):
    mac_addr = mac_addr.replace(":", "").upper()
    first_hex = int(mac_addr[0:2], 16)
    return first_hex & 0x02 != 0


def interpolate_point(timestamp, timestamps, breakpoints):
    if timestamp <= timestamps[0]:
        print("timestamp too small: {} <= {}".format(timestamp, timestamps[0]))
        return breakpoints[0]
    if timestamp >= timestamps[-1]:
        print("timestamp too large: {} >= {}".format(
            timestamp, timestamps[-1]))
        return breakpoints[-1]

    for idx in range(len(timestamps)-1):
        if timestamps[idx] <= timestamp <= timestamps[idx+1]:
            return [breakpoints[idx][coor_id] + (timestamp - timestamps[idx]) /
                    (timestamps[idx+1] - timestamps[idx]) *
                    (breakpoints[idx+1][coor_id] - breakpoints[idx][coor_id])
                    for coor_id in [0, 1]]

In [ ]:
# First step: load file and get observation points
def load_file(filename, observation_ids, observation_sign, file_type, prune, th):
    print (filename)
    max_id = 0
    if observation_ids:
        max_id = max(max_id, int(observation_ids[-1][0][1:]))

    with open(filename, "r") as f_in:
        while True:
            line = f_in.readline().rstrip(" \n")
            if not line:
                break
            if file_type == "path":
                if line.startswith("Start:"):
                    device_id = os.path.basename(filename)[:2]
                    try:
                        breakpoints = [[float(coor) for coor in item.split(
                            ",")] for item in f_in.readline().rstrip(" \n").split(" ")]
                        timestamps = [
                            int(ts) for ts in f_in.readline().rstrip(" \n").split(" ")]
                    except ValueError:
                        break
                    continue
                timestamp, rssi_pairs = line.split(" ", 1)
                ground_truth = interpolate_point(
                    int(timestamp), timestamps, breakpoints)
            elif file_type == "db":
                device_id = os.path.basename(filename)
                coor, rssi_pairs = line.split(" ", 1)
                ground_truth = [float(item) for item in coor.split(",")]
            elif file_type == "new":
                wifi_json = json.loads(line)
                timestamp = wifi_json['sysTimeMs']
                rssi_pairs = ''
                for item in wifi_json['data']:
                    rssi_pairs += str(item['bssid'].replace(':','')) + ',' + str(item['rssi']) + ' '
                rssi_pairs = rssi_pairs.strip(' ')
                ground_truth = [None,None]
                device_id = None
            rssi_dict = {}
            for rssi_pair in rssi_pairs.split(" "):
                try:
                    mac = rssi_pair.split(",")[0]
                    rssi = rssi_pair.split(",")[1]
                    # remove virtual mac
                    if prune and is_virtual_mac(mac):
                        continue
                    if float(rssi) >= th:
                        rssi_dict[mac] = float(rssi)
                except ValueError:
                    continue
                except IndexError:
                    continue
            if rssi_dict:
                observation_ids.append(
                    ["{}{}".format(observation_sign, max_id+1), device_id, ground_truth, rssi_dict])
                max_id += 1

    print("{} loaded".format(filename))
    return observation_ids

In [ ]:
# Second step: generate ap nodes from given observation points
def generate_ap_ids(ap_file, ap_sign, observation_ids):
    max_id = 0
    if os.path.isfile(ap_file):
        ap_ids = file_to_series(ap_file)
        max_id = max(max_id, int(ap_ids[-1][0][1:]))
    else:
        ap_ids = []

    for observation in observation_ids:
        for mac in observation[3].keys():
            ap_flag = False
            for ap in ap_ids:
                if ap[1] == mac:
                    ap_flag = True
                    ap[3].append([observation[3][mac]]+observation[2])
                    break
            if not ap_flag:
                ap_ids.append(["{}{}".format(ap_sign, max_id+1), mac,
                               None, [[observation[3][mac]]+observation[2]]])
                max_id += 1
    if ap_ids:
        series_to_file(ap_ids, ap_file)
    else:
        print("No AP ids!")
    return ap_ids

In [ ]:
# Third step: use observation nodes and ap nodes to generate edgelist for further use
def generate_graph_file(observation_file, ap_file, offset, output_file):
    observation_ids = file_to_series(observation_file)
    ap_ids = file_to_series(ap_file)
    ap_dict = {}
    for ap_item in ap_ids:
        ap_dict[ap_item[1]] = ap_item[0]

    with open(output_file, "w") as f_out:
        for observation in observation_ids:
            ob_id = observation[0]
            for mac in observation[3].keys():
                f_out.write("{} {} {}\n".format(ob_id, ap_dict[mac], rssi2weight(
                    offset, observation[3][mac])))
    print("edgelist generated.")

settings_id = "preprocess"
settings_file = "/content/drive/MyDrive/Floors/settings.json"

if __name__ == "__main__":
    # load settings
    offset, ap_sign, observation_sign, th = 0, "", "", 0
    with open(settings_file, 'r') as f_in:
        all_settings = json.load(f_in)
        settings = all_settings[settings_id]
        offset = settings['offset']
        ap_sign = settings['ap sign']
        observation_sign = settings['observation sign']
        th = settings['threshold']
        input_dir_name = settings["node output dir"]
        output_dir_name = settings["graph output parent dir"]
        target_dir_name = settings["graph output dir"]

    print("processing...")
    info_dict = {}
    #root_dir = os.getcwd()
    root_dir = "/content/drive/MyDrive/Floors/"
    input_dir = os.path.join(root_dir, input_dir_name)
    output_dir = os.path.join(root_dir, output_dir_name)
    # reset the output directory
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    # else:
    #     shutil.rmtree(output_dir)
    #     os.mkdir(output_dir)

    for raw_data_dir in os.listdir(input_dir):
        item = os.path.join(input_dir, raw_data_dir)
        if raw_data_dir not in building_ids:
            continue
        if os.path.isdir(item):
            output_dir_dataset = os.path.join(output_dir, raw_data_dir)
            target_dir = os.path.join(output_dir_dataset, target_dir_name)
            if not os.path.isdir(output_dir_dataset):
                os.mkdir(output_dir_dataset)
                os.mkdir(target_dir)
                os.mkdir(os.path.join(output_dir_dataset, "embedding"))
                os.mkdir(os.path.join(output_dir_dataset, "anchor list"))
                os.mkdir(os.path.join(output_dir_dataset, "spring"))
                os.mkdir(os.path.join(output_dir_dataset, "anchor map"))
                os.mkdir(os.path.join(output_dir_dataset, "incremental data"))

            observation_file = os.path.join(target_dir, "_observations_{}.pkl".format(raw_data_dir))
            ap_file = os.path.join(target_dir, "_APs_{}.pkl".format(raw_data_dir))

            pruned_observation_file = os.path.join(target_dir, "_observations_prune_{}.pkl".format(raw_data_dir))
            pruned_ap_file = os.path.join(target_dir, "_APs_prune_{}.pkl".format(raw_data_dir))

            graph_file = os.path.join(target_dir, "{}.edgelist".format(raw_data_dir))
            pruned_graph_file = os.path.join(target_dir, "prune_{}.edgelist".format(raw_data_dir))

            observation_ids = []
            pruned_observation_ids = []
            for filename in os.listdir(item):
                if filename.endswith("WiFi.txt"):
                    file = os.path.join(item, filename)
                    observation_ids = load_file(file, observation_ids, observation_sign, file_type="path", prune=False, th=th)
                    pruned_observation_ids = load_file(file, pruned_observation_ids, observation_sign, file_type="path", prune=True, th=th)
                elif filename.startswith("fingerprint"):
                    file = os.path.join(item, filename)
                    observation_ids = load_file(file, observation_ids, observation_sign, file_type="db", prune=False, th=th)
                    pruned_observation_ids = load_file(file, pruned_observation_ids, observation_sign, file_type="db", prune=True, th=th)
                elif filename.startswith("wifi-"):
                    file = os.path.join(item, filename)
                    observation_ids = load_file(file, observation_ids, observation_sign, file_type="new", prune=False, th=th)
                    pruned_observation_ids = load_file(file, pruned_observation_ids, observation_sign, file_type="new", prune=True, th=th)

            series_to_file(observation_ids, observation_file)
            series_to_file(pruned_observation_ids, pruned_observation_file)

            ap_ids = generate_ap_ids(ap_file, ap_sign, observation_ids)

            pruned_ap_ids = generate_ap_ids(pruned_ap_file, ap_sign, pruned_observation_ids)

            generate_graph_file(observation_file, ap_file, offset, graph_file)
            generate_graph_file(pruned_observation_file, pruned_ap_file, offset, pruned_graph_file)

            info_dict[raw_data_dir] = {}
            info_dict[raw_data_dir]["observation"] = len(observation_ids)
            info_dict[raw_data_dir]["ap"] = len(ap_ids)
            info_dict[raw_data_dir]["prune observation"] = len(pruned_observation_ids)
            info_dict[raw_data_dir]["prune ap"] = len(pruned_ap_ids)

    print("generation completed.")
    print(info_dict)

In [ ]:
#import os
#import pickle5 as pickle #pickle already imported
#import json
#rom const import *

#root_dir = os.getcwd()
root_dir = "/content/drive/MyDrive/Floors/"

for building_id in building_ids:
    raw_path = os.path.abspath(os.path.join(root_dir, "./data/output_data/{}/raw_data").format(building_id))
    obs_path = os.path.join(raw_path, "_observations_prune_{}.pkl".format(building_id))
    out_file_floors = os.path.abspath(os.path.join(root_dir, "./data/output_data/{}/raw_data/floors.txt".format(building_id)))

    with open(obs_path, 'rb') as f_in1, open(out_file_floors, "w") as f_out:
        series_obs = pickle.load(f_in1)
        series_obs = {series_obs[i][0]:series_obs[i][1]for i in range(len(series_obs))}
        f_out.write(str(series_obs))

In [ ]:
!python --version
print(tf. __version__)

In [ ]:
import random
import tensorflow as tf
from const import *
from graph import Graph
from eline import E_LINE
from paths import *
#from utils import *
os.environ['CUDA_VISIBLE_DEVICES']='0'

def clean_main(site_id, order, rep_size, negative_ratio):
    # step 1: preprocess, generate network edgelist for further processing
    site_folder = os.path.abspath(os.path.join(ROOT_FOLDER, "drive/MyDrive/Floors/", output_dir_graph, site_id))
    site_embedding_folder = os.path.join(site_folder, "embedding")
    observation_ids = file_to_series(os.path.join(site_folder, "raw_data", "_observations_{}.pkl".format(site_id)))
    ap_ids = file_to_series(os.path.join(site_folder, "raw_data", "_APs_{}.pkl".format(site_id)))
    pruned_observation_ids = file_to_series(os.path.join(site_folder, "raw_data", "_observations_prune_{}.pkl".format(site_id)))
    prunde_ap_ids = file_to_series(os.path.join(site_folder, "raw_data", "_APs_prune_{}.pkl".format(site_id)))

    # edgelist_file = gen_partial_edgelist_file(site_id, selected_ids, percent)
    edgelist_file = os.path.join(site_folder, "raw_data", "{}.edgelist".format(site_id))
    if not os.path.isfile(edgelist_file):
        generate_graph_file(edgelist_file, observation_ids, ap_ids)

    # step 2: E-LINE
    embedding_file = os.path.join(site_embedding_folder, "dim_{}.txt".format(rep_size))
    if not os.path.isfile(embedding_file):
        g = Graph()
        g.read_edgelist(filename=edgelist_file, weighted=True)
        model = E_LINE(g, epoch=100, rep_size=rep_size, order=order, negative_ratio=negative_ratio)
        model.save_embeddings(embedding_file)


if __name__ == "__main__":
    order = 2
    rep_size = 8
    negative_ratio = 4

    with tf.device('/GPU:0'):
        for idx0 in range(len(building_ids)):
            site_folder = os.path.abspath(os.path.join(ROOT_FOLDER, output_dir_graph, building_ids[idx0]))
            clean_main(building_ids[idx0], order, rep_size, negative_ratio)

In [ ]:
from hierarchical_centroid import *
from multiprocess import Pool
from const import *

dimension_of_vector = default_config['emb_dim']
num_reference_points_each_floor = default_config['ref_num_per_floor']


def main_hierarchical_average(reference_matrix, out_file):
    global ground_truth_value_list
    cluster_list = clustering_method_hierarchical_average_new(dataset, reference_matrix)
    pred_list = generate_pred_list_hierarchical_average(cluster_list, dataset, reference_matrix, ground_truth_value_list)
    print("reference matrix size is", reference_matrix.size)
    print("reference matrix is:\n", reference_matrix)
    print("pred list is:\n", pred_list)
    print("len of pred list is", len(pred_list))
    accuracy, report = assess_hierarchical_average(pred_list, ground_truth_list, ground_truth_value_list)
    print ("Out file: {}".format(out_file))
    with open(out_file, 'a') as f_out:
        f_out.write('\n'+str(report))
    return accuracy, pred_list


# Hierarchical test starts here ===================================================================
repeat_times = 10
acc_list = []
total_pred_list = []
total_truth_list = []


def util(i):
    global embedding_filename, ground_truth_filename, dataset, reference_matrix, ground_truth_value_list, ground_truth_list
    file_path = ''
    for building_id in building_ids:
        embedding_filename = out_file_embedding.format(building_id, dimension_of_vector)
        if not os.path.exists(embedding_filename):
            continue
        ground_truth_filename = out_file_floors.format(building_id)
        out_file_undirected = out_file_report.format(building_id, dimension_of_vector)
        dataset, desired_num_of_clusters, ground_truth_list, ground_truth_value_list, num_of_samples_in_each_floor = \
            get_dataset_and_truth_list_two(file_path, dimension_of_vector, embedding_filename, ground_truth_filename)
        print(len(dataset))
        print(len(ground_truth_list))

        dataset = np.array(dataset)
        reference_matrix = generate_reference_matrix(ground_truth_list, ground_truth_value_list,
                                                     num_reference_points_each_floor)
        main_hierarchical_average(reference_matrix, out_file_undirected)


t_start = time.time()


if __name__ == '__main__':
    p = Pool(repeat_times)
    list_of_pred_list = p.map(util, range(repeat_times))
    p.close()
    p.join()

t_end = time.time()
print("Total time is", (t_end - t_start)/60, "min")